In this notebook we will create the graph analysis for each of the agent's mode and traffic profiles per slice.

In [1]:
import sys
sys.path.insert(0, '../')

# # Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx # 复杂网络分析库，用于构建和可视化图结构
# 论文应用：创建关联规则网络图，展示决策-效果的关系网络
import ast
from graphviz import Digraph # 图形可视化库，用于绘制有向图
# 论文应用：创建决策流程的有向图，展示状态转移关系
%matplotlib inline
import plotly.graph_objects as go
from plotly.subplots import make_subplots # 创建多个子图的工具
from IPython.display import display # Jupyter显示工具，用于更好地展示DataFrame等对象

# Import Constatns  - 导入项目中定义的常量
from script.experiments_constants import AGENT_EXPERIMENT_INFO
from script.experiments_constants import ENV_KPI_NAME_LIST

from constants import PROJ_ADDR

# 导入项目自定义模块
from script.load_data import handle_data
# 导入数据加载函数，负责：
# 1. 检查数据缓存
# 2. 从原始日志文件加载数据
# 3. 返回清洗后的KPI数据和决策数据
# 使用示例：kpi_data, decision_data = handle_data(agent_info, user_number=6)
from script.symbolic_representation import create_symbolic_state_decision_matrix
# 导入符号化表示创建函数，这是SYMBXRL方法的核心
# 功能：将数值状态和决策转换为基于一阶逻辑的符号表示
# 论文应用：实现从连续数值空间到离散符号空间的转换

# Load and Show some results for each one of the agents

In this part we will load the data of one of the agents and visualize some of the numerical data.

加载并展示每个代理的一些结果
在本部分中，我们将加载其中一个智能体的数据，并对部分数值数据进行可视化处理。

## Graph Plotter Function Code【SRG】
绘图函数代码

In [ ]:
# Choose one of the agents: 'embb-trf1', 'embb-trf2', 'urllc-trf1', 'urllc-trf2'
# 选择一个智能体类型进行分析
agent = 'urllc-trf1'
# 这里选择了"urllc-trf1" - 超可靠低延迟通信流量类型1
# 论文意义：专注于分析URLLC（超可靠低延迟通信）类型的网络切片行为
# 与之前embb-trf2的区别：
# - eMBB (embb): 增强移动宽带 - 注重高吞吐量
# - URLLC (urllc): 超可靠低延迟通信 - 注重可靠性和低延迟
# 可选值说明：
# - 'embb-trf1': 增强移动宽带 - 流量类型1 (高吞吐量场景)
# - 'embb-trf2': 增强移动宽带 - 流量类型2 (高吞吐量场景)
# - 'urllc-trf1': 超可靠低延迟通信 - 流量类型1 (高可靠性场景)
# - 'urllc-trf2': 超可靠低延迟通信 - 流量类型2 (高可靠性场景)

# Choose one of the users: 3, 4, 5, 6 - Combination of agent and user will determine a specific experiment
# 选择用户数量：3, 4, 5, 6 - 智能体和用户数量的组合唯一确定一个实验场景
user = 3
# 这里选择了3个用户的场景，代表较低的网络负载
# 论文意义：研究在低用户密度条件下URLLC服务的性能特征
# 与之前user=6的区别：
# - user=6: 高负载场景，测试系统压力性能
# - user=3: 低负载场景，测试基础服务质量
# 注释说明：每个(agent, user)组合对应一组特定的实验数据

# Assuming AGENT_EXPERIMENT_INFO, handle_data and create_symbolic_state_decision_matrix are defined elsewhere in your codebase
# 假设AGENT_EXPERIMENT_INFO、handle_data和create_symbolic_state_decision_matrix已在代码库的其他地方定义
# 这些模块和函数通过前面的import语句导入

# 获取选定智能体的详细实验配置信息
agent_info = AGENT_EXPERIMENT_INFO[agent]
# 加载和处理实验数据 - 数据预处理阶段
kpi_data, decision_data = handle_data(agent_info, user)
# 创建符号化状态决策矩阵 - SYMBXRL核心算法步骤
symbolic_df, marker_df = create_symbolic_state_decision_matrix(kpi_data, decision_data, agent_info, user)
# 合并决策列，创建统一的决策符号表示
symbolic_df['combined_decision'] = symbolic_df.apply(lambda row: f"{row['prb_decision']} - {row['sched_decision']}", axis=1)

# 打印当前分析场景信息
print(f"Agent: {agent} - Users: {user}")
print("")  # 输出空行，使控制台输出更清晰易读

## Graph Plotter Util【EE-KG分析】
图形绘制工具实用程序

In [83]:
def plot_graph_from_data(df, column_name, output_path, node_threshold=0, edge_threshold=0, title_suffix=''):
    """
    从数据绘制决策状态转移图
    论文应用：可视化决策序列的马尔可夫过程，展示状态转移模式
    """
    # 计算状态转移概率矩阵（行归一化）
    cross_data = pd.crosstab(df[column_name], df[column_name].shift(-1), normalize='index') * 100
     # 功能：创建当前状态到下一状态的转移概率矩阵
    # df[column_name].shift(-1): 将数据向下移动一行，得到下一时间步的状态
    # normalize='index': 按行归一化，每行的概率和为100%
    # * 100: 转换为百分比

    # 计算每个状态的频率（节点大小基础）
    decision_counts = df[column_name].value_counts(normalize=True)
    # 返回每个决策状态的相对频率（概率）
    
    # Filter nodes based on node_threshold
    # 基于节点阈值过滤节点（去除低频状态）
    filtered_nodes = decision_counts[decision_counts >= node_threshold].index
    # node_threshold: 节点频率阈值，只保留频率≥该值的节点
    # 论文意义：简化网络图，突出重要模式
    
    # Filter edges based on edge_threshold and filtered nodes
    # 基于边阈值过滤边，并只保留过滤后的节点
    filtered_cross_data = cross_data.loc[filtered_nodes, filtered_nodes]
    # 只保留过滤后节点之间的转移关系

    # 进一步过滤边：只保留概率≥edge_threshold的转移
    for i, row in filtered_cross_data.iterrows():
        filtered_cross_data.loc[i] = row[row >= edge_threshold * 100]
        # 将低于阈值的转移概率设为NaN（后续处理为0）
    
    # Normalize the filtered cross data to ensure the probabilities sum to 1
    # 重新归一化过滤后的转移概率矩阵，确保每行和为100%
    filtered_cross_data = filtered_cross_data.div(filtered_cross_data.sum(axis=1), axis=0).fillna(0) * 100
    # .div(..., axis=0): 按行除法，重新归一化
    # .fillna(0): 将NaN值替换为0
    # * 100: 恢复百分比表示
    
    # 创建Graphviz有向图对象
    dot = Digraph(comment='Decision Graph', engine='dot')
    # engine='dot': 使用dot布局引擎（层次布局）

     # 计算图形尺寸（基于节点和边数量动态调整）
    num_nodes = len(filtered_nodes)
    num_edges = len(filtered_cross_data.columns)
    
    # Set the size of the graph based on the number of nodes and edges
    width = max(12, num_nodes * 2) # 最小宽度12，随节点数增加
    height = max(8, num_edges) # 最小高度8，随边数增加
    dot.attr(rankdir='LR', size=f'{width},{height}', dpi='120', bgcolor='white')
    # rankdir='LR': 从左到右布局（Left to Right）
    # size: 图形尺寸（英寸）
    # dpi: 输出分辨率
    # bgcolor: 背景颜色

    # Add nodes
    # 添加节点到图中
    for node, freq in decision_counts.items():
        if node in filtered_nodes:  # 只添加过滤后的节点
            node_size = 1 + 2 * freq  # Moderate size difference  # 节点大小与频率成正比（适度差异）
             # 构建节点标签：分割决策组合 + 显示概率
            label = node.split(' - ') # 分割PRB决策和调度决策
            label.append(f'prob: {freq:.1%}')  # 添加概率信息
            label = '\n'.join(label) # 用换行符连接
            # 创建节点
            dot.node(node, label, shape='ellipse', 
                     width=str(node_size), height=str(node_size),
                     style='filled', fillcolor='#E6F3FF', color='#4A6FE3',
                     fontname='Arial', fontsize='10')
            # 参数说明：
            # - shape='ellipse': 椭圆形节点
            # - width/height: 节点尺寸
            # - style='filled': 填充样式
            # - fillcolor: 填充颜色（浅蓝色）
            # - color: 边框颜色（深蓝色）
            # - fontname/size: 字体设置
    
    # Add edges
    # 添加边（状态转移）到图中
    for i, row in filtered_cross_data.iterrows(): # i: 源状态
        for j, prob in row.items(): # j: 目标状态, prob: 转移概率
            if prob > 0: # 只添加概率>0的边
                penwidth = 0.5 + prob / 50  # Reduced edge width scaling  # 边宽度与转移概率成正比（缩放因子减小）
                # 创建有向边
                dot.edge(i, j,
                         label=f'{prob:.1f}%',  # 显示转移概率
                         penwidth=str(penwidth),  # 边宽度
                         color='#4A6FE3',        # 边颜色
                         fontname='Arial', fontsize='8',
                         fontcolor='#4A6FE3')    # 标签颜色
    
    # Add legend with title suffix
     # 添加图例说明
    dot.attr(label=f'Node size: state frequency | Edge width: transition probability | {title_suffix}', 
             fontname='Arial', fontsize='12', labelloc='t') # labelloc='t': 标题位于顶部
    
    # Save the graph as a PDF
    # 保存图形为PDF文件
    dot.render(output_path, format='pdf', cleanup=True)
     # cleanup=True: 渲染后清理临时文件
    
    return dot # 返回图形对象，便于进一步操作或显示

## Create Graphs
执行函数，创建图表并保存

In [ ]:
# Plot the graph for each slice, display in the notebook, and save as PDF
# 为每个切片绘制图形，在notebook中显示并保存为PDF
slices = [0, 1, 2]# 三个网络切片ID
# 论文意义：分别分析每个切片的决策模式，比较差异化策略

# 设置输出目录路径【A1-NetworkSlicingSchedulingPolicy/results/decision_graphs/embb-trf1-users3/Decision_Graph_embb-trf1_user-3_slice-0.pdf】
output_dir = f"{proj_address}/A1-NetworkSlicingSchedulingPolicy/results/decision_graphs/{agent}-users{user}"  # Specify the directory to save the graphs


# 遍历每个切片，分别生成决策状态转移图
for slice_id in slices:
    # Filter the data for the current slice
    # 过滤出当前切片的数据
    slice_data = symbolic_df[symbolic_df['slice_id'] == slice_id].copy()
    # 使用.copy()避免修改原始数据，确保数据安全
    # 论文意义：分离分析不同切片的决策行为
    
    # Generate the graph with a title suffix indicating the slice number
    # 生成图形，标题后缀指明切片编号
    title_suffix = f'Slice {slice_id}' # 标题后缀，如 "Slice 0"
    output_path = f"{output_dir}/Decision_Graph_{agent}_user-{user}_slice-{slice_id}"
     # 文件命名规范：决策图_智能体_用户数_切片号
    # 示例：Decision_Graph_urllc-trf1_user-3_slice-0
    # 论文意义：标准化命名便于文件管理和引用
    
    # 调用绘图函数生成决策状态转移图
    dot = plot_graph_from_data(slice_data,              # 当前切片的数据
                              'combined_decision',      # 使用的列名（组合决策）
                              output_path,              # 输出文件路径
                              node_threshold=0.01,      # 节点频率阈值1%
                              edge_threshold=0.01,      # 边概率阈值1%
                              title_suffix=title_suffix) # 标题后缀
    # 阈值参数说明：
    # - node_threshold=0.01: 只显示出现频率≥1%的决策状态
    # - edge_threshold=0.01: 只显示转移概率≥1%的状态转移
    # 论文意义：过滤噪声，突出显著模式，提高图形可读性
    
    # Display the graph inline in the Jupyter notebook
    # 在Jupyter notebook中内联显示图形
    display(dot)
    # 功能：即时可视化反馈，便于交互式分析
    # 论文意义：支持探索性数据分析过程

# 完成提示信息
print("Decision graphs have been generated, saved as PDFs, and displayed inline.")
# 确认所有图形已生成、保存并显示